In [2]:
class Node:
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

class StackFrontier:
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            return self.frontier.pop()

class QueueFrontier(StackFrontier):
    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [3]:
def shortest_path(source, target, people, movies):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.
    If no possible path, returns None.
    """
    start = Node(state=source, parent=None, action=None)
    frontier = QueueFrontier()
    frontier.add(start)

    explored = set()

    while not frontier.empty():
        node = frontier.remove()

        if node.state == target:
            path = []
            while node.parent is not None:
                path.append((node.action, node.state))
                node = node.parent
            path.reverse()
            return path

        explored.add(node.state)

        for movie_id, person_id in neighbors_for_person(node.state, people, movies):
            if not frontier.contains_state(person_id) and person_id not in explored:
                child = Node(state=person_id, parent=node, action=movie_id)
                if child.state == target:
                    path = [(child.action, child.state)]
                    while child.parent is not None:
                        child = child.parent
                        if child.parent is not None:
                            path.append((child.action, child.state))
                    path.reverse()
                    return path
                frontier.add(child)

    return None

In [4]:
def neighbors_for_person(person_id, people, movies):
    """
    Returns (movie_id, person_id) pairs for people who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for star_id in movies[movie_id]["stars"]:
            if star_id != person_id:
                neighbors.add((movie_id, star_id))
    return neighbors

In [5]:
import csv

names = {}
people = {}
movies = {}

def load_data(directory):
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            name_key = row["name"].lower()
            if name_key not in names:
                names[name_key] = {row["id"]}
            else:
                names[name_key].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

In [6]:
def person_id_for_name(name):
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            print(f"ID: {person_id}, Name: {person['name']}, Birth: {person['birth']}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [8]:
directory = "large"  
load_data(directory)

source_name = input("Name: ")
target_name = input("Name: ")

source = person_id_for_name(source_name)
target = person_id_for_name(target_name)

if source is None or target is None:
    print("Person not found.")
else:
    path = shortest_path(source, target, people, movies)
    if path is None:
        print("Not connected.")
    else:
        print(f"{len(path)} degrees of separation.")
        for i, (movie_id, person_id) in enumerate(path):
            movie = movies[movie_id]["title"]
            person = people[person_id]["name"]
            print(f"{i + 1}: {person} starred in {movie}")

FileNotFoundError: [Errno 2] No such file or directory: 'large/people.csv'

In [10]:
!pip3 install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [11]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 8888-643B

 Directory of C:\Users\saras

05/26/2025  03:37 PM    <DIR>          .
05/26/2025  12:28 PM    <DIR>          ..
01/29/2024  05:44 PM            31,987  jadi python.ipynb
05/15/2024  07:44 PM           170,531  presentation.ipynb
09/14/2022  03:11 PM    <DIR>          .anaconda
03/28/2024  08:06 PM    <DIR>          .cache
10/13/2022  07:09 PM    <DIR>          .conda
09/14/2022  03:11 PM                25 .condarc
09/14/2022  03:10 PM    <DIR>          .continuum
12/18/2023  09:30 AM             1,543 .dbshell
10/01/2024  03:40 PM             6,148 .DS_Store
11/09/2022  07:09 PM               126 .gitconfig
07/31/2023  12:57 PM               176 .gitignore
09/15/2022  04:07 PM    <DIR>          .idlerc
05/26/2025  12:30 PM    <DIR>          .ipynb_checkpoints
09/14/2022  03:00 PM    <DIR>          .ipython
10/08/2022  04:10 PM    <DIR>          .jupyter
07/30/2023  01:22 PM    <DIR>          .keras
09/25/2024  06:24 PM    <D

In [14]:
!pip3 install -r 


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-r option requires 1 argument
